<a href="https://colab.research.google.com/github/Tiger-A/GoogleCollab/blob/main/transformers_fs_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color='steelblue' size=5><b>Вспомогательные материалы</b></font>

In [ ]:
# установим последнюю версию scikit-learn, если она не установлена
# теперь для OneHotEncoder работает одновремеено drop='first' и handle_unknown='ignore'
!pip install scikit-learn==1.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 32.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
# помним про PEP-8
# импорты из стандартной библиотеки
import warnings

# импорты сторонних библиотек
import numpy as np
import pandas as pd

# импорты модулей текущего проекта
# длина строки до 78 символов
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (
    GridSearchCV, 
    RandomizedSearchCV,
    train_test_split
)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler
)

# настройки
warnings.filterwarnings("ignore")

# константы заглавными буквами
RANDOM_STATE = 42

In [ ]:
def get_data_info(data):
    display(data.sample(5))
    display(data.info())
    display(data.describe(include='all'))

In [ ]:
df = pd.read_csv('/content/fs_autos.csv')
get_data_info(df)

,brand,model,registration_year,kilometer,vehicle_type,gearbox,power,fuel_type,repaired,price
118,mercedes_benz,clk,2001,169895,coupe,auto,218,gas,0,3999
38132,volkswagen,golf,2005,163612,coupe,manual,200,petrol,0,8300
29924,opel,tigra,2005,76999,convertible,auto,90,petrol,0,5600
40063,audi,a8,2003,171988,sedan,auto,334,petrol,0,9500
28344,mercedes_benz,e_klasse,2001,162666,sedan,auto,170,petrol,0,3000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   brand              50000 non-null  object
 1   model              50000 non-null  object
 2   registration_year  50000 non-null  int64 
 3   kilometer          50000 non-null  int64 
 4   vehicle_type       50000 non-null  object
 5   gearbox            50000 non-null  object
 6   power              50000 non-null  int64 
 7   fuel_type          50000 non-null  object
 8   repaired           50000 non-null  int64 
 9   price              50000 non-null  int64 
dtypes: int64(5), object(5)
memory usage: 3.8+ MB


None

,brand,model,registration_year,kilometer,vehicle_type,gearbox,power,fuel_type,repaired,price
count,50000,50000,50000.000000,50000.00000,50000,50000,50000.000000,50000,50000.000000,50000.000000
unique,6,83,NaN,NaN,8,2,NaN,3,NaN,NaN
top,volkswagen,golf,NaN,NaN,sedan,manual,NaN,petrol,NaN,NaN
freq,13023,5203,NaN,NaN,17700,34829,NaN,48315,NaN,NaN
mean,NaN,NaN,2005.789240,141167.59590,NaN,NaN,149.636700,NaN,0.074220,7678.924040
std,NaN,NaN,3.614953,40269.71589,NaN,NaN,50.756171,NaN,0.262131,4890.753177
min,NaN,NaN,2000.000000,5035.00000,NaN,NaN,80.000000,NaN,0.000000,235.000000
25%,NaN,NaN,2003.000000,128448.75000,NaN,NaN,110.000000,NaN,0.000000,3650.000000
50%,NaN,NaN,2006.000000,157261.00000,NaN,NaN,140.000000,NaN,0.000000,6700.000000
75%,NaN,NaN,2008.000000,168799.25000,NaN,NaN,170.000000,NaN,0.000000,10900.000000


<font color='steelblue' size=3><b>Предположим, что датасет уже изучен. Далее для начала необходимо разделить датасет на тренировочную и тестовую выборки.</b></font>

In [ ]:
features = df.drop(['price'], axis=1)
target = df['price']

X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.25, random_state=RANDOM_STATE
)


In [ ]:
get_data_info(X_train)

,brand,model,registration_year,kilometer,vehicle_type,gearbox,power,fuel_type,repaired
40647,opel,corsa,2004,177072,small,manual,80,petrol,0
27397,volkswagen,golf,2004,151480,sedan,manual,105,petrol,0
10294,audi,tt,2008,90785,convertible,manual,250,petrol,0
34786,audi,a3,2009,77610,sedan,auto,200,petrol,0
5097,bmw,5er,2004,166579,sedan,auto,218,petrol,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 37500 entries, 27434 to 15795
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   brand              37500 non-null  object
 1   model              37500 non-null  object
 2   registration_year  37500 non-null  int64 
 3   kilometer          37500 non-null  int64 
 4   vehicle_type       37500 non-null  object
 5   gearbox            37500 non-null  object
 6   power              37500 non-null  int64 
 7   fuel_type          37500 non-null  object
 8   repaired           37500 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 2.9+ MB


None

,brand,model,registration_year,kilometer,vehicle_type,gearbox,power,fuel_type,repaired
count,37500,37500,37500.000000,37500.000000,37500,37500,37500.000000,37500,37500.000000
unique,6,82,NaN,NaN,8,2,NaN,3,NaN
top,volkswagen,golf,NaN,NaN,sedan,manual,NaN,petrol,NaN
freq,9704,3888,NaN,NaN,13279,26023,NaN,36254,NaN
mean,NaN,NaN,2005.779600,141306.811600,NaN,NaN,149.909600,NaN,0.074720
std,NaN,NaN,3.610091,40163.113807,NaN,NaN,51.052166,NaN,0.262943
min,NaN,NaN,2000.000000,5092.000000,NaN,NaN,80.000000,NaN,0.000000
25%,NaN,NaN,2003.000000,128901.750000,NaN,NaN,110.000000,NaN,0.000000
50%,NaN,NaN,2006.000000,157285.000000,NaN,NaN,140.000000,NaN,0.000000
75%,NaN,NaN,2008.000000,168741.250000,NaN,NaN,170.000000,NaN,0.000000


In [ ]:
get_data_info(X_test)

,brand,model,registration_year,kilometer,vehicle_type,gearbox,power,fuel_type,repaired
43384,volkswagen,golf,2000,159826,sedan,manual,105,petrol,0
42102,opel,vivaro,2007,163249,bus,manual,101,petrol,0
46510,audi,a3,2002,156936,sedan,manual,131,petrol,0
30506,audi,a3,2010,63288,small,manual,105,petrol,0
25082,volkswagen,passat,2007,167457,sedan,manual,105,petrol,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12500 entries, 33553 to 37270
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   brand              12500 non-null  object
 1   model              12500 non-null  object
 2   registration_year  12500 non-null  int64 
 3   kilometer          12500 non-null  int64 
 4   vehicle_type       12500 non-null  object
 5   gearbox            12500 non-null  object
 6   power              12500 non-null  int64 
 7   fuel_type          12500 non-null  object
 8   repaired           12500 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 976.6+ KB


None

,brand,model,registration_year,kilometer,vehicle_type,gearbox,power,fuel_type,repaired
count,12500,12500,12500.000000,12500.000000,12500,12500,12500.0000,12500,12500.000000
unique,6,82,NaN,NaN,8,2,NaN,3,NaN
top,volkswagen,golf,NaN,NaN,sedan,manual,NaN,petrol,NaN
freq,3319,1315,NaN,NaN,4421,8806,NaN,12061,NaN
mean,NaN,NaN,2005.818160,140749.948800,NaN,NaN,148.8180,NaN,0.072720
std,NaN,NaN,3.629489,40586.592157,NaN,NaN,49.8507,NaN,0.259687
min,NaN,NaN,2000.000000,5035.000000,NaN,NaN,80.0000,NaN,0.000000
25%,NaN,NaN,2003.000000,127167.500000,NaN,NaN,110.0000,NaN,0.000000
50%,NaN,NaN,2006.000000,157142.500000,NaN,NaN,140.0000,NaN,0.000000
75%,NaN,NaN,2008.000000,168919.500000,NaN,NaN,170.0000,NaN,0.000000


<font color='steelblue' size=3><b>Предположим, что выборки не требуют никакой обработки аномалий, выбросов и т. д., логично перейти к кодированию и масштабированию.<br>
Допустим, мы решили, что одной из моделей будет Ridge, для котрой будем использовать OHE кодирование категориальных признаков и StandardScaler для численных признаков. Второй моделью будет RandomForestRegressor, для которой будет использоваться порядковое кодирование признака model, имеющего большое количество уникальных значений, и OHE кодирование остальных категориальных признаков.</b></font>

In [ ]:
#категориальные признаки для OHE Ridge
ohe_features_ridge = X_train.select_dtypes(include='object').columns.to_list()
print(ohe_features_ridge)

#категориальные признаки для OHE RandomForestRegressor
ohe_features_rf = ohe_features_ridge.copy()
ohe_features_rf.remove('model')
ohe_features_rf

['brand', 'model', 'vehicle_type', 'gearbox', 'fuel_type']


['brand', 'vehicle_type', 'gearbox', 'fuel_type']

In [ ]:
#численные признаки
#обратите внимание, что 'repaired' — категориальный бинарный признак.
num_features = X_train.select_dtypes(exclude='object').columns.to_list()
num_features.remove('repaired')
num_features

['registration_year', 'kilometer', 'power']

<font color='steelblue' size=3><b>
Начнём с линейной модели.<br><br>
Имеется нюанс: в процессе изучения данных было обнаружено, что и для тренировочной, и для тестовой выборок у признака 'model' 82 уникальных значения, но в тренировочной выборке имеется значения 'calibra', которого нет в тестовой выборке, а в тестовой выборке имеется значение 'up', которого нет в тренировочной. Это совершенно обычная для данных ситуация, которую можно легко пропустить, ведь количество признаков абсолютно одинаковое.</b></font>

In [ ]:
models_train = set(X_train['model'].unique())
models_test = set(X_test['model'].unique())
num_models_train = len(models_train)
num_models_test = len(models_test)
print(f'''
Количество уникальных значений признка "model" 
в обеих выборках одинаковое: {num_models_train == num_models_test}
''')
print(f'''
Уникальные значения признка "model" 
в обеих выборках одинаковые: {models_train == models_test}
''')
print(f'''
Только в тренировочной выборке есть значения: {models_train - models_test}
''')
print(f'''
Только в тестовой выборке есть значения: {models_test - models_train}
''')


Количество уникальных значений признка "model" 
в обеих выборках одинаковое: True


Уникальные значения признка "model" 
в обеих выборках одинаковые: False


Только в тренировочной выборке есть значения: {'calibra'}


Только в тестовой выборке есть значения: {'up'}



<font color='steelblue' size=3><b>
Если говорить о прямом кодировании, использование get_dummies в такой ситуации пройдёт без падающего кода, вы просто преобразуете выборки, отдадите их модели, количество признаков одинаковое, а значит код нигде не упадёт. Но в кодировании будет ошибка, ведь в тренировочной и тестовой выборках уникальные значения признака не совпадают.<br><br>
Дело в том, что get_dummies не запоминает информацию о признаках, он скорее подходит для анализа. Конечно можно найти эту особенность, после кодирования удалить из тестовой выборки признак 'up', добавить в неё признак 'calibra' c 0 значениями, тогда всё будет корректно. Предполагается, что модель будет использоваться множество раз, для каждой новой тестовой выборки придётся так же смотреть на каждый признак и делать эту предобработку. Можно даже написать функцию, которая, автоматизирует эти действия. В этом случае можно будет поздравить автора, он изобрёл велосипед 😅 Все эти нюансы уже учтены в OneHotEncoder, давайте посмотрим на него поближе:</b></font>

In [ ]:
X_train_ridge = X_train.copy()
X_test_ridge = X_test.copy()

In [ ]:
# drop='first' удаляет первый признак из закодированных:
# таким образом обходим dummy-ловушку
# задаём handle_unknown='ignore':
# игнорируется ранее невстречающиеся значения признака (при transform)
encoder_ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)

# обучаем энкодер на заданных категориальных признаках тренировочной выборки
encoder_ohe.fit(X_train_ridge[ohe_features_ridge])

# добавляем закодированные признаки в X_train_ohe
# encoder_ohe.get_feature_names_out() позволяет получить названия колонок
X_train_ridge[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(X_train_ridge[ohe_features_ridge])

# удаляем незакодированные категориальные признаки (изначальные колонки)
X_train_ridge = X_train_ridge.drop(ohe_features_ridge, axis=1)

# создаём скелер
scaler = StandardScaler()

# обучаем его на численных признаках тренировочной выборки, трансформируем её же
X_train_ridge[num_features] = scaler.fit_transform(X_train_ridge[num_features])

# смотрим на результат
X_train_ridge.head()

,registration_year,kilometer,power,repaired,brand_bmw,brand_ford,brand_mercedes_benz,brand_opel,brand_volkswagen,model_3er,...,vehicle_type_convertible,vehicle_type_coupe,vehicle_type_other,vehicle_type_sedan,vehicle_type_small,vehicle_type_suv,vehicle_type_wagon,gearbox_manual,fuel_type_other,fuel_type_petrol
27434,-1.600978,0.569390,0.001771,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
13400,1.446077,0.778466,0.393532,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
883,0.338057,0.371868,-0.879692,0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7303,0.892067,-1.627904,-0.135346,0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
45124,0.892067,0.954276,1.451288,0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


<font color='steelblue' size=3><b>
Если решите остановиться на этом способе и Ridge окажется лучшей моделью, будем трансформировать тестовую выборку так:</b></font>

In [ ]:
# энкодером, который обучен на ТРЕНИРОВОЧНОЙ ВЫБОРКЕ, кодируем тестовую
X_test_ridge[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(X_test_ridge[ohe_features_ridge])

X_test_ridge = X_test_ridge.drop(ohe_features_ridge, axis=1)

# скелером, который обучен на ТРЕНИРОВОЧНОЙ ВЫБОРКЕ, масштабируем тестовую
X_test_ridge[num_features] = scaler.transform(
    X_test_ridge[num_features]
)

# смотрим на результат
X_test_ridge.head()

,registration_year,kilometer,power,repaired,brand_bmw,brand_ford,brand_mercedes_benz,brand_opel,brand_volkswagen,model_3er,...,vehicle_type_convertible,vehicle_type_coupe,vehicle_type_other,vehicle_type_sedan,vehicle_type_small,vehicle_type_suv,vehicle_type_wagon,gearbox_manual,fuel_type_other,fuel_type_petrol
33553,-0.492958,0.519842,-0.370403,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
9427,0.061052,0.507666,1.607992,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
199,-1.323973,0.885306,1.333759,0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
12447,-0.492958,0.857669,-0.958045,0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
39489,-0.492958,0.302326,-1.173513,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [ ]:
# получилось одинаковое количество признаков
X_train_ridge.shape, X_test_ridge.shape

((37500, 100), (12500, 100))

In [ ]:
# получились одинаковые признаки в тренировочной и тестовой выборках
(X_train_ridge.columns != X_test_ridge.columns).sum()

0

<font color='steelblue' size=3><b>
Если более элегантное решения для кодирования и масштабирования, попробуем make_column_transformer:</b></font>

In [ ]:
X_train_ridge = X_train.copy()
X_test_ridge = X_test.copy()

In [ ]:
# признак repaired уже бинарный, его не будем кодировать/масштабировать
# добавляем remainder='passthrough, чтобы он не пропал 
col_transformer_ridge = make_column_transformer(
    (
        OneHotEncoder(drop='first', handle_unknown='ignore'),
        ohe_features_ridge
    ),
    (
        StandardScaler(), 
        num_features
    ),
    remainder='passthrough',
    verbose_feature_names_out=False
)

# всё готово в пару строк кода
X_train_ridge = pd.DataFrame.sparse.from_spmatrix(
    col_transformer_ridge.fit_transform(X_train_ridge),
    columns=col_transformer_ridge.get_feature_names_out()
)

# смотрим на результат
X_train_ridge.head()

,brand_bmw,brand_ford,brand_mercedes_benz,brand_opel,brand_volkswagen,model_3er,model_5er,model_6er,model_7er,model_a1,...,vehicle_type_small,vehicle_type_suv,vehicle_type_wagon,gearbox_manual,fuel_type_other,fuel_type_petrol,registration_year,kilometer,power,repaired
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,-1.600978,0.569390,0.001771,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.446077,0.778466,0.393532,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.338057,0.371868,-0.879692,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.892067,-1.627904,-0.135346,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.892067,0.954276,1.451288,0.0


<font color='steelblue' size=3><b>
Если решите остановиться на этом способе и Ridge окажется лучшей моделью, будем трансформировать тестовую выборку так:</b></font>

In [ ]:
# трансформируем тестовую выборку
X_test_ridge = pd.DataFrame.sparse.from_spmatrix(
    col_transformer_ridge.transform(X_test_ridge),
    columns=col_transformer_ridge.get_feature_names_out()
)

# смотрим на результат
X_test_ridge.head()

,brand_bmw,brand_ford,brand_mercedes_benz,brand_opel,brand_volkswagen,model_3er,model_5er,model_6er,model_7er,model_a1,...,vehicle_type_small,vehicle_type_suv,vehicle_type_wagon,gearbox_manual,fuel_type_other,fuel_type_petrol,registration_year,kilometer,power,repaired
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,-0.492958,0.519842,-0.370403,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.061052,0.507666,1.607992,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,-1.323973,0.885306,1.333759,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,-0.492958,0.857669,-0.958045,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,-0.492958,0.302326,-1.173513,0.0


<font color='steelblue' size=3><b>
Теперь подбираем гиперпараметры с помощью GridSearchCV, который имеет встроенную кросс-валидацию для модели Ridge. Лучший вариант — использовать пайплайны, это предотвратит возможность утечки на кросс-валидации. О них в самом конце, а пока рассморим вариант без него:</b></font>

In [ ]:
# random_state не перебирается, задаём его прямо в модели
model_ridge = Ridge(random_state=RANDOM_STATE)

# словарь с гиперпараметрами и значениями, которые хотим перебрать
param_grid_ridge = {
    'alpha': np.arange(0, 0.21, 0.01),
}

gs_ridge = GridSearchCV(
    model_ridge, 
    param_grid=param_grid_ridge, 
    scoring='neg_root_mean_squared_error', 
    n_jobs=-1
)

gs_ridge.fit(X_train_ridge, y_train)

# лучшее значение RMSE на кросс-валидации
print(f'best_score: {gs_ridge.best_score_ * -1}')

# лучшие гиперпараметры
print(f'best_params: {gs_ridge.best_params_}')

best_score: 2260.558448452929
best_params: {'alpha': 0.01}


<font color='steelblue' size=3><b>
Ещё более элегантным решением будет использовать пайплайн вместе с кросс-валидацией. Это позволит кодировать и масштабировать отдельно каждую тренировочную и валидационную выборки внутри кросс-валидации (без «подглядывания»):</b></font>

In [ ]:
X_train_ridge = X_train.copy()

In [ ]:
pipeline_ridge = make_pipeline(col_transformer_ridge, model_ridge)

param_grid_ridge = {
    'ridge__alpha': np.arange(0, 0.21, 0.01),
}

gs_ridge_pl = GridSearchCV(
    pipeline_ridge, 
    param_grid=param_grid_ridge, 
    scoring='neg_root_mean_squared_error', 
    n_jobs=-1
)

gs_ridge_pl.fit(X_train_ridge, y_train)

gs_ridge_best_score = gs_ridge_pl.best_score_ * -1
gs_ridge_best_params = gs_ridge_pl.best_params_

# лучшее значение RMSE на кросс-валидации
print(f'best_score: {gs_ridge_best_score}')
# лучшие гиперпараметры
print(f'best_params: {gs_ridge_best_params}')

best_score: 2260.559847084961
best_params: {'ridge__alpha': 0.02}


<font color='steelblue' size=3><b>
Перейдём к RandomForestRegressor:</b></font>

In [ ]:
X_train_rf = X_train.copy()
X_test_rf = X_test.copy()

In [ ]:
col_transformer_rf= make_column_transformer(
    (
        OneHotEncoder(drop='first', handle_unknown='ignore'), 
        ohe_features_rf
    ),
    (
        OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), 
        ['model']
    ),
    (
        StandardScaler(), 
        num_features
    ),
    remainder='passthrough'
)

model_rf = RandomForestRegressor(random_state=RANDOM_STATE)

pipline_rf = make_pipeline(col_transformer_rf, model_rf)

param_grid_rf = {
    'randomforestregressor__n_estimators': range(50, 251, 50),
    'randomforestregressor__max_depth': range(2, 15),
    'randomforestregressor__min_samples_split': (2, 3, 4),
    'randomforestregressor__min_samples_leaf': (1, 2, 3, 4)
}

# получается достаточно много комбинаций гиперпараметров при переборе
# будем использовать RandomizedSearchCV, он работает на много быстрее
gs_rf = RandomizedSearchCV(
    pipline_rf, 
    param_distributions=param_grid_rf, 
    scoring='neg_root_mean_squared_error', 
    n_jobs=-1, 
    random_state=RANDOM_STATE
)

gs_rf.fit(X_train_rf, y_train)

gs_rf_best_score = gs_rf.best_score_ * -1
gs_rf_best_params = gs_rf.best_params_
print(f'best_score: {gs_rf_best_score}')
print(f'best_params: {gs_rf_best_params}')

best_score: 1947.8392253783488
best_params: {'randomforestregressor__n_estimators': 250, 'randomforestregressor__min_samples_split': 4, 'randomforestregressor__min_samples_leaf': 3, 'randomforestregressor__max_depth': 12}


In [ ]:
result = pd.DataFrame(
    [gs_ridge_best_score, gs_rf_best_score], 
    index=['Ridge', 'RandomForestRegressor'], 
    columns=['RMSE']
)
result

,RMSE
Ridge,2260.559847
RandomForestRegressor,1947.839225


<font color='steelblue' size=3><b>
Лучшей моделью оказалась RandomForestRegressor. Проверим её качесво на тестовой выборке:</b></font>

In [ ]:
rf_prediction = gs_rf.predict(X_test_rf)
metric_test = mean_squared_error(y_test, rf_prediction, squared=False)
metric_test

1938.67987460236

<img src="https://www.storemypic.com/images/2016/11/22/keep-calm-and-good-luck-5f5ac.jpg">